### 查看GPU

In [1]:
!nvidia-smi

Sun Mar  3 10:23:20 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   47C    P8              14W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

### 安裝需要的BERT套件

In [2]:
!pip install transformers==4.29.0 -U
!pip install nlp -U
!pip install torch==2.0.0 -U
!pip install transformers accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 22.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 44.4 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.15.2
    Uninstalling tokenizers-0.15.2:
      Successfully uninstalled tokenizers-0.15.2
  Attempting uninstall: transformers
    Found existing installation: transformers 4.38.2
    Uninstalling transformers-4.38.2:
      Successfully uninstalled transformers-4.38.2


### 載入套件

In [3]:
from transformers import BertForSequenceClassification, BertTokenizerFast, Trainer, TrainingArguments
from nlp import load_dataset, Dataset
import nlp
import torch
import random
import pandas as pd
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

### 抓取訓練資料

In [4]:
!wget https://github.com/shhuangmust/AI/raw/112-1/train.csv

--2024-03-03 10:24:26--  https://github.com/shhuangmust/AI/raw/112-1/train.csv
Resolving github.com (github.com)... 192.30.255.112
Connecting to github.com (github.com)|192.30.255.112|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/shhuangmust/AI/112-1/train.csv [following]
--2024-03-03 10:24:26--  https://raw.githubusercontent.com/shhuangmust/AI/112-1/train.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 388895 (380K) [text/plain]
Saving to: ‘train.csv.2’

train.csv.2         100%[===================>] 379.78K  --.-KB/s    in 0.02s   

2024-03-03 10:24:26 (15.3 MB/s) - ‘train.csv.2’ saved [388895/388895]



### 讀取訓練資料集

In [5]:
df = pd.read_csv('train.csv')
dataset = Dataset.from_pandas(df)

In [6]:
df.sample(10)

,ID,Name,Description,label
431,431,長庚研發葉黃素明亮專案,「為天然的抗氧化劑，能保護眼睛細胞免於自由基的傷害，人體吸收率高達100%，可有效預防飛蚊症...,1
901,901,https://tw.buy.yahoo.com/gdsale/GNC健安喜-專利茄紅素-紅...,每顆含有番茄萃取物100毫克。-取自天然成熟紅番茄，富含天然茄紅素、β-胡蘿蔔素、植物固醇等...,0
160,160,宏醫雙效抗氧化健康咖啡組,「陳湘青口述：『...酸就是開始肥胖跟癌症...平均值一個禮拜幾乎可以到4-5公斤...藥局...,1
532,532,長庚純正牛樟菇菌絲體純液搶購組,「…牛樟菇菌絲體防止肝臟發炎作用明顯增進肺癌抑制療效並緩化療藥物的肝臟毒性…牛樟菇含有…促進...,1
549,549,MANUKA蜂蜜、MANUKA15+蜂蜜、MANUKA24+蜂蜜,「…對腸胃非常好…這等級的是藥用都不過份…腸胃不好更可以…糖尿病人吃蜂蜜比吃糖好…」,1
83,83,《日本美白專利》O-LIFE雪薊白青春錠,「日本美白專利，30秒喚醒白皙…想變白、容易曬黑、肌膚暗沉、老化者的救星…對抗肌膚黑斑、暗沉...,1
660,660,愛尚它?益生消化酵素粉末,我們吃進的食物，需仰賴各種的消化酵素來分解。愛尚它?益生消化酵素粉末幫助提供某些酵素（澱粉酵...,0
35,35,「青春不老丹」、「泛美頂級奈米珍珠粉」及「珍珠粉」,「台灣『最頂級奈米』珍珠粉..延緩老化美化肌膚...使肌膚細嫩光澤...珍珠粉可安神(安定神...,1
419,419,卡布速纖白咖啡,「(老公)老婆我們公司舉辦員工減重大賽，我是部門裡最胖的不能當害群之馬，明天你開始幫我帶便當...,1
157,157,維他露澳洲神果KKD青纖錠口碑囤貨檔,「杜詩梅口述：...因為在花蓮我都穿運動服所以沒有太大感受都很舒服很寬鬆然後準備要來上節目的...,1


### 載入原始模型

In [7]:
model = BertForSequenceClassification.from_pretrained('hfl/chinese-bert-wwm-ext')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of the model checkpoint at hfl/chinese-bert-wwm-ext were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClas

###  載入分詞模型

In [8]:
tokenizer = BertTokenizerFast.from_pretrained('hfl/chinese-bert-wwm-ext')

### 設定執行參數

In [9]:
RANDOM_SEED = 5
MAX_LEN = 512
EPOCHS=5
BATCH_SIZE = 8

### 把資料集分成訓練、測試、驗證

In [10]:
shuffled_ds = dataset.shuffle(RANDOM_SEED)
split_ds = shuffled_ds.train_test_split(test_size=0.2)
train_dataset = split_ds['train']
test_val_dataset = split_ds['test']
split_tv = test_val_dataset.train_test_split(test_size=0.5)
test_dataset = split_tv['train']
val_dataset = split_tv['test']

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

### 取一筆出來瞧瞧(已經打亂)

In [11]:
train_dataset[2]

{'Description': '「預防包括病毒、細菌、逆轉濾過性病毒、真菌、寄生蟲、酵母和黴菌在內的多種致病微生物的感染...支持免疫系統功能...橄欖葉提取物有助於促進細胞的健康環境」',
 'ID': 442,
 'Name': '【美優純】橄欖葉萃取膠囊OliveLeaf(120顆)',
 'label': 1}

In [12]:
tokenizer.tokenize("體重跟體脂都有在持續緩慢的下降中")

['體',
 '重',
 '跟',
 '體',
 '脂',
 '都',
 '有',
 '在',
 '持',
 '續',
 '緩',
 '慢',
 '的',
 '下',
 '降',
 '中']

### 定義分詞器

In [13]:
def tokenize(batch):
  return tokenizer(batch['Description'], max_length=MAX_LEN, padding=True, truncation=True)

train_dataset = train_dataset.map(tokenize, batched=True, batch_size=len(train_dataset))
test_dataset = test_dataset.map(tokenize, batched=True, batch_size=len(test_dataset))
val_dataset = val_dataset.map(tokenize, batched=True, batch_size=len(val_dataset))

train_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])
test_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])
val_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

### 定義評測準則，訓練器

In [14]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=EPOCHS,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    warmup_steps=50,
    weight_decay=0.01,
    # evaluate_during_training=True,
    logging_dir='./logs',
)

trainer = Trainer(
    model=model,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)

### 開始訓練

In [15]:
trainer.train()
trainer.evaluate()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss


{'eval_loss': 0.09034715592861176,
 'eval_accuracy': 0.9893617021276596,
 'eval_f1': 0.9911504424778761,
 'eval_precision': 1.0,
 'eval_recall': 0.9824561403508771,
 'eval_runtime': 2.0798,
 'eval_samples_per_second': 45.196,
 'eval_steps_per_second': 5.77,
 'epoch': 5.0}

### 找一個範例來測試

In [16]:
print(df.iloc[220]['label'], df.iloc[220]['Description'])
print(df.iloc[578]['label'], df.iloc[578]['Description'])

1 「增強免疫力...護眼明目...抗氧化...保肝...皮膚：...保護皮膚，減緩皮屑問題，降低敏感...毛髮……促進毛髮亮麗...消化系統」「強化免疫系統，有助調整過敏體質...清除自由基延緩老化...調整腸胃道機能...調理皮膚問題...諾麗果含有許多強大的抗氧化劑」「阻止或減少病痛的發生，並迅速修補已受傷的細胞，將代謝殘渣或毒素排出體外」「它能維護身體細胞組織正常運作，阻止或減少病痛的發生，並迅速修補已受傷的細胞，將代謝殘渣或毒素排出體外，使身體恢復正常」
0 ·???????每份含有30mg輔Q10。輔Q10可參與維生素E再生，兩者具協同作用，可促進新陳代謝。保護循環順暢維持健康。暢銷日本的最佳美容聖品。


### 設定測試文字

In [17]:
#test_text = '增強免疫力...護眼明目...抗氧化...保肝...皮膚：...保護皮膚，減緩皮屑問題，降低敏感...毛髮……促進毛髮亮麗...消化系統」「強化免疫系統，有助調整過敏體質...清除自由基延緩老化...調整腸胃道機能...調理皮膚問題...諾麗果含有許多強大的抗氧化劑」「阻止或減少病痛的發生，並迅速修補已受傷的細胞，將代謝殘渣或毒素排出體外」「它能維護身體細胞組織正常運作，阻止或減少病痛的發生，並迅速修補已受傷的細胞，將代謝殘渣或毒素排出體外，使身體恢復正常'
test_text = '???????每份含有30mg輔Q10。輔Q10可參與維生素E再生，兩者具協同作用，可促進新陳代謝。保護循環順暢維持健康。暢銷日本的最佳美容聖品'

MAX_LEN = 512

encoded_review = tokenizer.encode_plus(
    test_text,
    max_length=MAX_LEN,
    add_special_tokens=True,
    return_token_type_ids=False,
    return_attention_mask=True,
    return_tensors='pt',
    truncation=True
)

In [18]:
encoded_review

{'input_ids': tensor([[  101,   136,   136,   136,   136,   136,   136,   136,  3680,   819,
          1419,  3300,  8114,  8882,  6737, 11382,   511,  6737, 11382,  1377,
          1347,  5645,  5204,  4495,  5162,   147,  1086,  4495,  8024,  1060,
          5442,  1072,  1295,  1398,   868,  4500,  8024,  1377,   914,  6868,
          3173,  7376,   807,  6342,   511,   924,  6362,  2542,  4472,  7518,
          3268,  5204,  2898,   978,  2434,   511,  3268,  7077,  3189,  3315,
          4638,  3297,   881,  5401,  2159,  5469,  1501,   102]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

### 進行測試

In [20]:
test_model = model.to('cuda')

input_ids = encoded_review['input_ids'].to('cuda')
attention_mask = encoded_review['attention_mask'].to('cuda')
output = test_model(input_ids, attention_mask)
result = torch.argmax(output[0][0])

classnames = ['合格', '不合格']

print(f'廣告詞: {test_text}')
print(f'檢測結果: {classnames[result]}')

廣告詞: ???????每份含有30mg輔Q10。輔Q10可參與維生素E再生，兩者具協同作用，可促進新陳代謝。保護循環順暢維持健康。暢銷日本的最佳美容聖品
檢測結果: 合格
